In [7]:
import pandas as pd
import os 
import psycopg2
import logging
import ast
absolutepath = os.path.abspath(os.getcwd())
fileDirectory = os.path.dirname(absolutepath)
parentDirectory = os.path.dirname(fileDirectory)
dataprocDirectory = os.path.join(fileDirectory, 'data/proc')
datarawDirectory = os.path.join(fileDirectory, 'data/raw')

In [8]:
# Read the cleaned data
jttp_cleaned = pd.read_csv(os.path.join(dataprocDirectory, 'jttp_data_cleaned.csv'))

# Create 'year' column by copying 'recruit_year'
jttp_cleaned['year'] = jttp_cleaned['recruit_year']

# Use ast.literal_eval to safely evaluate string representations of lists
jttp_cleaned['author_id_eval'] = [
    ast.literal_eval(v) if isinstance(v, str) else v  # Apply only if the value is a string
    for v in jttp_cleaned['author_id'].values
]

# Drop rows with NaN values in 'author_id_eval'
jttp_cleaned = jttp_cleaned.dropna(subset=['author_id_eval'])

# Extract the first element from each evaluated list (i.e., the first author_id)
jttp_cleaned['JTTP_authid'] = [v[0] for v in jttp_cleaned['author_id_eval'].values]


## merge in subfield
### copying the field file
cp /cluster/work/lawecon/Work/dcai/jiycai/thousand_talent/thousand_talent/scholar_fields.csv /cluster/work/lawecon/Work/dcai/jiycai/thousand_talent/thousand_talent/scripts/10152022_iteration_peer_effect/data/raw

In [11]:
scholar_field = pd.read_csv(os.path.join(datarawDirectory, 'scholar_fields.csv'))

In [12]:
jttp_cleaned = jttp_cleaned.merge(scholar_field, how = 'left', left_on = ['JTTP_authid'], right_on = ['authid'])

In [13]:
del scholar_field

# create treatment timing dataframe
## each row is (aff,2 digit field, 4 digit field, year of first jttp, fraction never returned among first impact, num jttp joining among first impact,)
## merge in subfield

In [14]:
groupby_variables_4digit = ['hiring_unit_id','subfield_most_frequent',
                            'subfield_most_frequent_two_digit']
outcome_variables_4digit = ['hiring_unit_id','subfield_most_frequent',
                            'subfield_most_frequent_two_digit',
                            'year','never_returned','JTTP_authid']
jttp_cleaned['first_impact_4digit_time'] = jttp_cleaned[outcome_variables_4digit].\
groupby(groupby_variables_4digit).transform('min')[['year']]
afid_subfield_timing = jttp_cleaned.loc[jttp_cleaned.year == jttp_cleaned.first_impact_4digit_time,
                                        outcome_variables_4digit].groupby(groupby_variables_4digit).\
agg({'year':'min','never_returned':'mean','JTTP_authid':'nunique'}).reset_index()


In [15]:
afid_subfield_timing['reneger_treatment_4digit'] = 1*(afid_subfield_timing['never_returned']==1)

In [16]:
afid_subfield_timing.never_returned.value_counts()

never_returned
0.000000    1421
1.000000     227
0.500000      34
0.250000       5
0.333333       3
0.666667       2
0.200000       1
Name: count, dtype: int64

In [17]:
afid_subfield_timing.JTTP_authid.value_counts()

JTTP_authid
1    1517
2     138
3      27
4      10
5       1
Name: count, dtype: int64

In [18]:
afid_subfield_timing[afid_subfield_timing.JTTP_authid == 5]

,hiring_unit_id,subfield_most_frequent,subfield_most_frequent_two_digit,year,never_returned,JTTP_authid,reneger_treatment_4digit
1435,60032356,_2500,_25,2017,0.2,5,0


# what are the aff X 4 digit with 5 incoming at first impact?
## Beijing Computational Science Research Center
## Physical Sciences; Physics and Astronomy; Condensed Matter Physics;3104
### select * from scopus_affiliation where afid = 60104686;

## Shanghai Institute for Biological Sciences Chinese Academy of Sciences
## Life Sciences;Agricultural and Biological Sciences;Plant Science; 1110
### select * from scopus_affiliation where afid = 60007324;


#### https://github.com/plreyes/Scopus/blob/master/ASJC%20Codes%20with%20levels.csv

In [19]:
groupby_variables_2digit = ['hiring_unit_id',
                            'subfield_most_frequent_two_digit']
outcome_variables_2digit = ['hiring_unit_id',
                            'subfield_most_frequent_two_digit',
                            'year','never_returned','JTTP_authid']
jttp_cleaned['first_impact_2digit_time'] = jttp_cleaned[outcome_variables_2digit].\
groupby(groupby_variables_2digit).transform('min')[['year']]
afid_subfield_timing_2digit = jttp_cleaned.loc[jttp_cleaned.year == jttp_cleaned.first_impact_2digit_time,\
                                               outcome_variables_2digit].groupby(groupby_variables_2digit).\
agg({'year':'min','never_returned':'mean','JTTP_authid':'nunique'}).reset_index()

In [20]:
afid_subfield_timing_2digit['reneger_treatment_2digit'] = 1*(afid_subfield_timing_2digit['never_returned']==1)

In [21]:
afid_subfield_timing_2digit.never_returned.value_counts()

never_returned
0.000000    807
1.000000    125
0.500000     36
0.250000      5
0.200000      2
0.333333      2
0.666667      1
Name: count, dtype: int64

In [22]:
afid_subfield_timing_2digit.JTTP_authid.value_counts()

JTTP_authid
1    812
2    123
3     25
4     14
5      4
Name: count, dtype: int64

# save treatment timing files

In [23]:
afid_subfield_timing.to_csv(os.path.join(dataprocDirectory, 'afid_subfield_timing_4digit.csv'),index=False)
afid_subfield_timing_2digit.to_csv(os.path.join(dataprocDirectory, 'afid_subfield_timing_2digit.csv'),index=False)

# create list of ever treated schools

In [24]:
ever_treated_schools = jttp_cleaned.hiring_unit_id.unique()

# update the jttp_school column in scopus_affiliation

UPDATE scopus_affiliation
SET jttp_school = 0;

# create query

In [25]:
condition = " or ".join([f"afid = {afid}" for afid in ever_treated_schools])
query = f"""UPDATE scopus_affiliation SET jttp_school = 1 WHERE ({condition});"""

In [26]:
query

'UPDATE scopus_affiliation SET jttp_school = 1 WHERE (afid = 60010851 or afid = 60105683 or afid = 60007324 or afid = 60013789 or afid = 60000937 or afid = 60009860 or afid = 60006782 or afid = 60019706 or afid = 60019118 or afid = 60017060 or afid = 60005465 or afid = 60018704 or afid = 60029306 or afid = 60016521 or afid = 60025278 or afid = 60018205 or afid = 60122781 or afid = 60006354 or afid = 60019533 or afid = 60014773 or afid = 60033100 or afid = 60069717 or afid = 60032955 or afid = 60003970 or afid = 60017705 or afid = 60073652 or afid = 60019499 or afid = 60025084 or afid = 120075110 or afid = 60024045 or afid = 60011069 or afid = 60022422 or afid = 60014966 or afid = 60031863 or afid = 60025256 or afid = 60105232 or afid = 60026601 or afid = 60021170 or afid = 60014277 or afid = 60026282 or afid = 60010432 or afid = 60019616 or afid = 60013551 or afid = 60031031 or afid = 60028265 or afid = 60024542 or afid = 60015742 or afid = 60102070 or afid = 60010896 or afid = 6011341

# create list of ever treated fields

In [29]:
ever_treated_fields = jttp_cleaned.subfield_most_frequent_two_digit.unique()

# update the jttp_field column in scopus_authors_aff_year_filled_chinese

ALTER TABLE scopus_authors_aff_year_filled_chinese ADD COLUMN jttp_field float;
UPDATE scopus_authors_aff_year_filled_chinese
SET jttp_field = 0;

In [30]:
condition = " or ".join([f"subfield_most_frequent_two_digit = '{field}'" for field in ever_treated_fields])
query = f"""UPDATE scopus_authors_aff_year_filled_chinese SET jttp_field = 1 WHERE ({condition});"""

In [34]:
query

"UPDATE scopus_authors_aff_year_filled_chinese SET jttp_field = 1 WHERE (subfield_most_frequent_two_digit = '_23' or subfield_most_frequent_two_digit = '_16' or subfield_most_frequent_two_digit = '_11' or subfield_most_frequent_two_digit = '_27' or subfield_most_frequent_two_digit = '_22' or subfield_most_frequent_two_digit = '_31' or subfield_most_frequent_two_digit = '_13' or subfield_most_frequent_two_digit = '_28' or subfield_most_frequent_two_digit = '_19' or subfield_most_frequent_two_digit = '_26' or subfield_most_frequent_two_digit = '_24' or subfield_most_frequent_two_digit = 'nan' or subfield_most_frequent_two_digit = '_17' or subfield_most_frequent_two_digit = '_25' or subfield_most_frequent_two_digit = '_21' or subfield_most_frequent_two_digit = '_30' or subfield_most_frequent_two_digit = '_10' or subfield_most_frequent_two_digit = '_33' or subfield_most_frequent_two_digit = '_15' or subfield_most_frequent_two_digit = '_29' or subfield_most_frequent_two_digit = '_18' or sub

In [33]:
jttp_cleaned.subfield_most_frequent_two_digit.count()

np.int64(3525)